# _Beacon Runner 2050: One beacon, many runners_

###### May 2020, [@barnabemonnot](https://twitter.com/barnabemonnot)
###### [Robust Incentives Group](https://github.com/ethereum/rig), Ethereum Foundation
###### Built with specs v0.11.1

---

## TL;DR

- We improve upon our second [_Beacon Runner 2049_](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2049/beacon_runner_2049.ipynb), an economics-focused simulation environment for eth2.
- Network latencies and communications are fully represented, with each validator storing their current view of the chain or incoming blocks and attestations in a `Store` object defined in the [specs](https://github.com/ethereum/eth2.0-specs/blob/dev/specs/phase0/fork-choice.md#store).
- Validator behaviours are fully modular and can be plugged to the simulation as long as they follow a simple API.

---

We want to understand how validator behaviours map to rewards, penalties and chain outcomes. Ideally, validators who are rational are also honest, i.e., they run the eth2 protocol the way it "should" be run. But apart from how incentives are designed, there is no guarantee that this will indeed the case. And as we will also see, "honesty" may not always have a unique instantiation.

In this notebook, we improve upon the [first](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner/beacon_runner.ipynb) and [second](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2049/beacon_runner_2049.ipynb) Beacon Runners by introducing a more full-fledged simulation environment.

### _Previously, on..._

In the first notebook, we introduced the possibility of "wrapping" the current specs in a [cadCAD](https://github.com/BlockScience/cadCAD) simulation environment. We defined simple _strategies_ for validators that allowed them to produce blocks and attest too. The implementation was "centralised" in the sense that all validators shared a common view of the chain at all time -- a situation akin to being on a network with _perfect information_ and _zero latency_.

The natural next step was to relax this assumption, and allow different views of the chain to coexist. In the simplest case, these views have an empty intersection: this is the case when the network is perfectly _partitioned_, and each side of the partition works independently. We explored there how the _inactivity leak_, which decreases the stake of inactive validators, eventually allows for finalisation to resume. But what if this intersection is not empty? In other words, what if some validators see both sides of the partition? More generally, what if each validator has their own view of the chain, governed by the messages they have received from other validators on the network?

These are the conditions we explore here. They are sufficient to represent an imperfect p2p network, where validators receive updates from each other after some (random) delay. We'll delve more into how the network is represented later, but you can also refer to the implementation presented in the second notebook.

### Getting started

Once again, we import the specs loaded with a custom configuration file, where epochs are only 4 slots long (for demonstration purposes).

In [1]:
%%capture
import specs
import importlib
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

prepare_config(".", "fast")
importlib.reload(specs)

We import our network library, seen in [network.py](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2050/network.py), as well as a library of helper functions for our Beacon Runners, [brlib.py](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2050/brlib.py). Open them up! The code is not that scary.

In [2]:
import network as nt
import brlib

Now on to the new stuff. We moved `honest_attest` and `honest_propose` to a new [validatorlib.py](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2050/validatorlib.py) file. This file also defines a very important class, the `BRValidator`, intended to be an abstract superclass to custom validator implementations. `BRValidator` comes packaged with a `Store`, a nifty little helper class defined in the [specs](https://github.com/ethereum/eth2.0-specs/blob/dev/specs/phase0/fork-choice.md#store) and a bunch more logic to record past actions and current parameters. We'll get to them in a short while. 

We intend `BRValidator` to be an abstract superclass, meaning that though it is not supposed to be instantiated, it is friendly to inheritance. Subclasses of `BRValidator` inherit its attributes and methods, and are themselves intended to follow a simple API. Subclasses of `BRValidator` must expose a `propose()` and an `attest()` method which return, respectively, a block or an attestation when queried (or `None` when they are shy and don't want to return anything yet). We provide an example implementation in [ASAPValidator.py](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2050/ASAPValidator.py) (a.k.a, _Slasho_), a very nice validator who always proposes and attests as soon as they can, and honestly too.

In [3]:
import validatorlib as vlib
from ASAPValidator import *

Let's talk about cadCAD once more. Our simulations are now stochastic, since the latency of the network means that some updates are random. cadCAD makes it easy to organise and run any number of instances as well as define the steps that take place in each instance. But our simulation state is pretty large: there are _n_ validators and for each validator, a certain amount of data to keep track of, including chain states and current unincluded blocks and attestations. With advice from [the cadCAD community](https://community.cadcad.org/t/mitigating-cadcad-overhead/140), and a nice "tweak" by [Danilo Lessa](https://twitter.com/danilolessa) to the source, the simulations are implemented following the pattern described [in this notebook](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2049/observers/observed-br2049.ipynb).

In short, make sure that you clone and checkout the `tweaks` branch of [Danilo's fork](https://github.com/danlessa/cadCAD/tree/tweaks). Install cadCAD in the folder besides this notebook, in editable mode (option `-e` with `pip3` or `pipenv`). Alternatively, refer to the [Pipfile](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2050/Pipfile) included here. If you install cadCAD from pypi or any other package manager, you will get the default version and the simulations will be quite a bit slower.

In [4]:
from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
import pandas as pd

Are we all set? It seems so!

## Discovering the validator and network APIs

We'll start slow, as we have done in previous notebooks, before moving to a bigger simulation. We loaded a specs configuration with 4 slots per epoch, so we'll instantiate 4 ASAP validators such that each will attest in a different slot.

### Genesis

First, we obtain a genesis state with 4 deposits registered. Second, we instantiate our validators from this state. A `Store` is created in each of them that records the genesis state root and a couple other things. Finally we ask our validators to skip the genesis block -- it is a special block at slot 0 that no one is supposed to suggest, the first block from a validator being expected at slot 1.

In [121]:
genesis_state = brlib.get_genesis_state(4, seed="riggerati")
validators = [ASAPValidator(genesis_state, i) for i in range(4)]
brlib.skip_genesis_block(validators)

>>> recomputing attester
>>> recomputing proposer
------ update_data
updated moves 0.00023412704467773438
updated received 0.00028324127197265625
got head 0.00037407875061035156
updated proposers/attesters 0.0003941059112548828
------ update_data 0.0004069805145263672
------ update_data
updated moves 0.00011396408081054688
updated received 0.00014901161193847656
>>> recomputing proposer
>>> recomputing attester
updated proposers/attesters 0.0023202896118164062
------ update_data 0.00234222412109375


Note that the current store time is exactly `SECONDS_PER_SLOT` ahead of `genesis_time` (in our configuration, and the current canonical specs, 12 seconds). We fast-forwarded beyond the first block at 0 to the start of slot 1.

In [122]:
print("Genesis time = ", validators[0].store.genesis_time)
print("Store time = ", validators[0].store.time)
print("Current slot = ", validators[0].data.slot)

Genesis time =  1578182400
Store time =  1578182412
Current slot =  1


Let's now reuse the network we had in [the second notebook](https://github.com/ethereum/rig/blob/master/eth2economics/code/beaconrunner2049/beacon_runner_2049.ipynb). The four validators are arranged along a chain, validator 0 peering with validator 1, which peers with validator 2, which peers with validator 3. We create information sets (who peers with who) to represent the chain.

In [123]:
set_a = nt.NetworkSet(validators=list([0,1]))
set_b = nt.NetworkSet(validators=list([1,2]))
set_c = nt.NetworkSet(validators=list([2,3]))

net = nt.Network(validators = validators, sets = list([set_a, set_b, set_c]))

### Proposer duties

When we instantiate new validators, as we have done with `ASAPValidator(genesis_state, validator_index)`, their constructor preloads a few things. First, each validator checks their proposer duties for all slots of the current epoch.

In [124]:
proposer_views = [(validator_index, validator.data.current_proposer_duties) \
                  for validator_index, validator in enumerate(net.validators)] 
proposer_views

[(0, [False, False, False, False]),
 (1, [True, True, False, False]),
 (2, [False, False, False, True]),
 (3, [False, False, True, False])]

The array above shows for each validator index (0, 1, 2, 3) whether they are expected to propose a block in either of the 4 slots. Notice that the randomness means the same validator could be called twice in an epoch. This is distinct from attestation duties, where each validator is expected to attest once, and only once, in each epoch.

Since we are at slot 1, we see that validator 1 is expected to propose here. Let's ping them by calling their `propose` method, which expects a dictionary of "known items": blocks and attestations communicated over the network which may not have been included in the chain yet. Validator 1 has not seen anything yet, so we'll just leave them empty.

In [125]:
block = net.validators[1].propose({ "attestations": [], "blocks": [] })

honest validator 1 propose a block for slot 1
block contains 0 attestations


Unsurprisingly, the new block produced does not contain any attestation.

Now validator 1 communicates its block to the information sets it belongs to, namely, to validators 0 and 2.

In [126]:
nt.disseminate_block(net, 1, block)

--------- disseminate_block
going to ask to check backlog 0.0003218650817871094
-------- ask_to_check_backlog
------- check_backlog
finished recording blocks 0.002662181854248047
finished recording attestations 0.0027441978454589844
------ update_data
updated moves 4.57763671875e-05
updated received 0.00010991096496582031
got head 0.0006258487701416016
keep searching
parent is old head
updated proposers/attesters 0.0007798671722412109
------ update_data 0.0008769035339355469
done updating data 0.0037031173706054688
------- end check_backlog 0.003731250762939453
-------- end ask_to_check_backlog 0.010045289993286133
--------- end disseminate_block 0.010515213012695312


Let's check who received the block. We obtained the hash of the block and check with validators 0 and 3 if they know of any block with this hash.

In [127]:
block_root = hash_tree_root(block.message)

try:
    net.validators[0].store.blocks[block_root]
    print("0: there is a block")
except KeyError: print("0: no block")
    
try:
    net.validators[3].store.blocks[block_root]
    print("3: there is a block")
except KeyError: print("3: no block")

0: there is a block
3: no block


This confirms that validator 3 has not seen the block yet. In the next network update, triggered when `update_net` is called on the current `network` object, validator 2 communicates the block to validator 3. But let's not do that just yet, and instead fast-forward a little more to slot number 2.

In [128]:
for validator in net.validators:
    validator.forward_by(specs.SECONDS_PER_SLOT)
print("Validator 0 says this is slot number", net.validators[0].data.slot)

------ update_data
updated moves 0.00014400482177734375
updated received 0.0003058910369873047
>>> recomputing proposer
>>> recomputing attester
updated proposers/attesters 0.0035851001739501953
------ update_data 0.0036249160766601562
Validator 0 says this is slot number 2


### Attester duties

Let's check who is expected to attest at slot 2. Our `BRValidator` superclass records the slot of the current epoch where validators are expected to attest, in a `current_attest_slot` attribute of their `data`. In general, computing attester or proposer duties is expensive, so we try to cache it when we can and recompute it only when necessary.

In [129]:
committee_views = [validator.data.current_attest_slot for validator in net.validators] 
committee_views

[0, 1, 2, 3]

At slot 2, validator 2 is expected to attest. Let's check what items they currently know about.

In [130]:
known_items = nt.knowledge_set(net, 2)
known_items

{'attestations': [],
 'blocks': [NetworkBlock(item=SignedBeaconBlock(Container)
      message: BeaconBlock = BeaconBlock(Container)
                                 slot: Slot = 1
                                 proposer_index: ValidatorIndex = 1
                                 parent_root: Root = 0x4fb1f050a706b5ebf74f40551ce55f42421e7fadc37da9df8ab7aecf40b2c22b
                                 state_root: Root = 0xc44c671ad230e0588f88c5eafb403f99834082e56d6aeb200845c9eb68e3e3b2
                                 body: BeaconBlockBody = BeaconBlockBody(Container)
                                                             randao_reveal: BLSSignature = 0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
                                                             eth1_data: Eth1Data = Eth1Data(Container)
                                        

Validator 2 knows about the block that validator 1 sent in slot 1! All is well here. Validator 2's attestation will set this block as the current head of the chain and the heat goes on.

In [131]:
attestation = net.validators[2].attest(known_items)
print(attestation)

None


Woah, what happened here? Validator 2 refused to attest.

Let's back up a bit and see why. Validator 2 is expected to attest during slot 2. Honest validators however are supposed to leave a bit of time for block proposers to communicate their blocks. We are indeed in slot 2, but we are early into it, at the very start. Meanwhile, slots last for about 12 seconds and validators are only expected to attest a third of the way into the slot, i.e., 4 seconds in. This leaves 4 seconds for the block proposer of slot 2 to produce their block and communicate it (in reality, a bit more since producers can start producing before the end of the previous slot, at the risk of missing incoming attestations).

Alright. Let's assume that no one wants to propose anything for this slot. We'll forward everyone by 4 seconds and see if validator 2 is ready to attest then.

In [132]:
for validator in net.validators:
    validator.forward_by(4)

In [133]:
print("Validator 2 says this is slot number", net.validators[2].data.slot)
print("Time is now", net.validators[2].store.time)
print("We are now",
      (net.validators[2].store.time - net.validators[2].store.genesis_time) % specs.SECONDS_PER_SLOT, "seconds into the slot")

Validator 2 says this is slot number 2
Time is now 1578182428
We are now 4 seconds into the slot


Ready to attest now?

In [134]:
attestation = net.validators[2].attest(known_items)
print(attestation)

attestation 0xfaec3db26c039be25eede638df8ee8ea96b599333ae5bf4a6f4494fcea415ff2 for slot 2 by validator 2 source 0 and target 0
Attestation(Container)
    aggregation_bits: SpecialBitlistView = Bitlist[2048](1 bits: 1)
    data: AttestationData = AttestationData(Container)
                                slot: Slot = 2
                                index: CommitteeIndex = 0
                                beacon_block_root: Root = 0xad38a973c0b920ba6d7058973646f16d04218b620b53513b01fcfad6dc67ef8e
                                source: Checkpoint = Checkpoint(Container)
                                                         epoch: Epoch = 0
                                                         root: Root = 0x0000000000000000000000000000000000000000000000000000000000000000
                                target: Checkpoint = Checkpoint(Container)
                                                         epoch: Epoch = 0
                                                         root:

Yes! Validator 2 returned a well-formed attestation. ASAP validators are in a hurry, sure, but not so into a hurry that they would attest too early in the slot.

What are the dangers of attesting too early? Simply put, validators are rewarded for attesting to the correct head of the chain as of the slot they are attesting for. If a validator attests too early and votes for the head in slot 1, after which the block for slot 2 is revealed and included in the canonical chain, the validator does not receive a reward for correctly voting on the head.

But shouldn't a validator attest as late as possible then? We are already entering the realm of game theory here. I like it. Maybe! Though attesting too late means that the reward obtained for being included early decreases, and if you are _really_ too late, like a few epochs late, then you are not included at all. So pick your poison here. As an exercise (with more details at the end) I'll leave it to you to write a validator behaviour that attests as soon as a block is received in the slot, or no later than the end of the slot if no block comes in. It's very easy! Let's talk about it later.

We need to forward a bit more for validators to deign record the new attestation. By default, validators ignore incoming attestations for the slot they are currently in. This is because an attestation for slot 2 can at the earliest be included in a block for slot 3. So let's jump to slot 3 by forwarding by 8 seconds.

In [135]:
for validator in net.validators:
    validator.forward_by(8)

------ update_data
updated moves 0.0002529621124267578
updated received 0.00032520294189453125
>>> recomputing proposer
>>> recomputing attester
updated proposers/attesters 0.003818035125732422
------ update_data 0.0038499832153320312


Let's have validator 2 disseminate their attestation.

In [136]:
nt.disseminate_attestations(net, [(2, attestation)])

--------- disseminate_attestations 1 attestations
going to ask to check backlog 0.00031113624572753906
-------- ask_to_check_backlog
------- check_backlog
finished recording blocks 5.4836273193359375e-05
finished recording attestations 0.001050710678100586
------ update_data
updated moves 4.1961669921875e-05
updated received 0.00011110305786132812
got head 0.0004830360412597656
updated proposers/attesters 0.0005197525024414062
------ update_data 0.0005478858947753906
done updating data 0.0017027854919433594
------- end check_backlog 0.0017290115356445312
-------- end ask_to_check_backlog 0.0031118392944335938
--------- end disseminate_attestations 0.003536224365234375


### Final state

We'll check the state of each validator in turn. The `store` records in its `latest_messages` attribute the latest message received from each other validators (message being attestation here). This is the LMD of LMD-GHOST, latest message-driven fork choice!

In [137]:
print(net.validators[0].store.latest_messages)

{}


Validator 0 has an empty `latest_messages` attribute. Remember that validator 0 is not peering with validator 2. Since the network was not updated, the recent attestation from validator 2 did not make its way to validator 0.

In [138]:
print(net.validators[1].store.latest_messages)

{2: LatestMessage(epoch=0, root=0xad38a973c0b920ba6d7058973646f16d04218b620b53513b01fcfad6dc67ef8e)}


Validator 1 has seen the attestation from validator 2, since they are peering together. This makes sense.

In [139]:
print(net.validators[2].store.latest_messages)

{2: LatestMessage(epoch=0, root=0xad38a973c0b920ba6d7058973646f16d04218b620b53513b01fcfad6dc67ef8e)}


Obviously, validator 2 also knows about its own attestation.

In [140]:
print(net.validators[3].store.latest_messages)

{}


Hmm, this is trickier. Validator 3 received validator 2's attestation, since they are peering together. But why isn't it showing here in the `latest_messages`?

The reason is simple: validator 2's attestation vouches for _validator 1's block_ as the current head of the chain. But validator 3 doesn't yet know about this block! From the point of view of validator 3, the attestation might as well be vouching for an inexistent head. In our `net` object, the attestation is recorded as "known" by validator 3, but it cannot participate in validator 3's fork choice, until validator 3 knows about validator 1's block.

So we have some intuition about what is going on behind the scene. Let's now take a look at a larger-scale simulation!

## Simulating a complete chain

In [19]:
%%capture

prepare_config(".", "medium")
importlib.reload(specs)
importlib.reload(nt)
importlib.reload(brlib)
importlib.reload(vlib)

In [20]:
num_validators = 30
genesis_state = brlib.get_genesis_state(num_validators)
validators = [vlib.ASAPValidator(genesis_state, i) for i in range(num_validators)]
brlib.skip_genesis_block(validators)

set_a = nt.NetworkSet(validators=list(range(0, int(num_validators * 2 / 3.0))))
set_b = nt.NetworkSet(validators=list(range(int(num_validators / 2.0), num_validators)))

network = nt.Network(validators = validators, sets=list([set_a, set_b]))

>>> recomputing attester
>>> recomputing proposer
------ update_data
updated moves 0.00012183189392089844
updated received 0.00016307830810546875
got head 0.0002529621124267578
updated proposers/attesters 0.0002727508544921875
------ update_data 0.0002868175506591797
------ update_data
updated moves 6.890296936035156e-05
updated received 0.00010919570922851562
>>> recomputing proposer
>>> recomputing attester
updated proposers/attesters 0.00817108154296875
------ update_data 0.008201122283935547


In [21]:
print("Set A = ", set_a)
print("Set B = ", set_b)

Set A =  NetworkSet(validators=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
Set B =  NetworkSet(validators=[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])


In [22]:
initial_conditions = {
    'network': network
}

block_attestation_psub = [
    {
        'policies': {
            'action': brlib.attest_policy
        },
        'variables': {
            'network': brlib.disseminate_attestations
        }
    },
    {
        'policies': {
            'action': brlib.propose_policy
        },
        'variables': {
            'network': brlib.disseminate_blocks
        }
    },
    {
        'policies': {
        },
        'variables': {
            'network': brlib.tick
        }
    },
]

In [23]:
number_slots = specs.SLOTS_PER_EPOCH + 1
steps = number_slots * specs.SECONDS_PER_SLOT * vlib.frequency

simulation_parameters = {
    'T': range(steps),
    'N': 1,
    'M': {}
}

print("will simulate", number_slots, "slots at frequency", vlib.frequency, "moves/second")
print("total", steps, "steps")

will simulate 17 slots at frequency 1 moves/second
total 204 steps


In [24]:
config = Configuration(initial_state=initial_conditions,
                       partial_state_update_blocks=block_attestation_psub,
                       sim_config=simulation_parameters
                      )

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results
df = pd.DataFrame(raw_result)


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: single_proc: [<cadCAD.configuration.Configuration object at 0x11f003700>]
Configurations: [<cadCAD.configuration.Configuration object at 0x11f003700>]
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
attest_policy time =  0.00047326087951660156
---------- disseminate_attestations brlib
--------- disseminate_attestations 0 attestations
going to ask to check backlog 3.910064697265625e-05
-------- ask_to_check_backlog
-------- end ask_to_check_backlog 1.1920928955078125e-05
--------- end disseminate_attestations 0.00011420249938964844
adding 0 to network items there are now 0 attestations
---------- end disseminate_attestations brlib 0.00023818016052246094
honest validator 5 propose a block for slot 1
block contains 0 attesta